In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
from torch import optim
import numpy as np
import math
import random
import sys
import os
import pickle

In [2]:
df = pd.read_csv('data/exps/rnn1/toy-1.csv')
df_binned = pd.read_csv('data/exps/rnn1/binned/toy.csv')

In [12]:
concat = pd.read_csv('data/exps/rnn1/toy_concat.csv')

In [17]:
for col in concat.columns:
    print(col)

unique_thread_id
fdbk_score_src
fdbk_pstv_src
offr_b0
slr_hist
byr_hist
byr_us
offr_s0
fdbk_score_start
fdbk_pstv_start
start_price_usd
photo_count
to_lst_cnt
bo_lst_cnt
ref_price1
view_item_count
wtchr_count
bin_rev
lstg_gen_type_id
store
slr_us
buyer_us
99m
220m
237m
260m
267m
281m
293m
316m
550m
619m
625m
870m
888m
1249m
1281m
1305m
2984m
3252m
10542m
11116m
11232m
11233m
11450m
11700m
12576m
14339m
15032m
20081m
26395m
45100m
58058m
64482m
172008m
1000c
1500c
1750c
2000c
2500c
2750c
3000c
4000c
5000c
6000c
7000c
duration
frac_remain_b0
frac_passed_b0
passed_b0
remain_b0
frac_remain_s0
frac_passed_s0
passed_s0
remain_s0
time_s0
time_b0
length
frac_passed_b1
frac_remain_b1
passed_b1
remain_b1
offr_b1
time_b1
frac_passed_s1
frac_remain_s1
passed_s1
remain_s1
offr_s1
time_s1
frac_passed_b2
frac_remain_b2
passed_b2
remain_b2
offr_b2
time_b2
frac_passed_s2
frac_remain_s2
passed_s2
remain_s2
offr_s2
time_s2
frac_passed_b3
frac_remain_b3
passed_b3
remain_b3
offr_b3
time_b3
ref_offr_b0
ref_

In [3]:
t = torch.tensor([[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]], [[10, 20, 30], [40, 50, 60], [70, 80, 90], [100, 110, 120]]])

In [5]:
linear = torch.nn.Linear(3, 6)
t.shape

3

In [6]:
t

tensor([[[   1,    2,    3],
         [   4,    5,    6],
         [   7,    8,    9],
         [  10,   11,   12]],

        [[  10,   20,   30],
         [  40,   50,   60],
         [  70,   80,   90],
         [ 100,  110,  120]]])

In [9]:
linear.weight
t = t.float()
lengths = [2, 2, 1, 1]


In [10]:
p = linear(t)
packed =  torch.nn.utils.rnn.pack_padded_sequence(t, torch.tensor([2, 2, 1, 1]))

In [11]:
l = torch.tensor([ 13.8407, -47.9710,  48.0697,  38.1333, -22.7242, 105.5340])
F.softmax(l, dim=0)

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000])

In [12]:
input = torch.randn(128, 3)
print(input.type())
print(t.type())
output = linear(input)
print(output.size())

torch.FloatTensor
torch.FloatTensor
torch.Size([128, 6])


In [25]:
linear(t.float().view(-1, 3))

tensor([[  0.0131,  -0.9351,   1.0980,   0.9514,  -1.1260,   2.0702],
        [  0.4343,  -2.2843,   2.3042,   1.8718,  -1.6210,   4.8225],
        [  0.8555,  -3.6335,   3.5104,   2.7922,  -2.1161,   7.5748],
        [  1.2767,  -4.9826,   4.7167,   3.7126,  -2.6111,  10.3271],
        [  1.2047,  -7.4956,  11.8827,  10.5209,  -7.8729,  22.9651],
        [  5.4167, -20.9874,  23.9450,  19.7250, -12.8233,  50.4881],
        [  9.6287, -34.4792,  36.0073,  28.9292, -17.7737,  78.0111],
        [ 13.8407, -47.9710,  48.0697,  38.1333, -22.7242, 105.5340]],
       grad_fn=<ThAddmmBackward>)

In [99]:
p

tensor([[[  0.0131,  -0.9351,   1.0980,   0.9514,  -1.1260,   2.0702],
         [  0.4343,  -2.2843,   2.3042,   1.8718,  -1.6210,   4.8225],
         [  0.8555,  -3.6335,   3.5104,   2.7922,  -2.1161,   7.5748],
         [  1.2767,  -4.9826,   4.7167,   3.7126,  -2.6111,  10.3271]],

        [[  1.2047,  -7.4956,  11.8827,  10.5209,  -7.8729,  22.9651],
         [  5.4167, -20.9874,  23.9450,  19.7250, -12.8233,  50.4881],
         [  9.6287, -34.4792,  36.0073,  28.9292, -17.7737,  78.0111],
         [ 13.8407, -47.9710,  48.0697,  38.1333, -22.7242, 105.5340]]],
       grad_fn=<ThAddBackward>)

In [13]:
max_length = 2
lengths = [2, 2, 1, 1]
lengths = np.array(lengths)
short_seqs = np.nonzero(lengths != max_length)
short_seqs = short_seqs[0]

In [14]:
short_lengths = lengths[lengths != max_length]
p.shape

torch.Size([2, 4, 6])

In [82]:
v = p.clone()
v[short_lengths, short_seqs, :] = 0
v[short_lengths, short_seqs, 0] = 1
v

In [20]:
crit =  torch.nn.CrossEntropyLoss(size_average=False, reduce=False, ignore_index=-100)

In [21]:
p = p.transpose(1, 2)

In [22]:
p

tensor([[[ -1.2678,  -1.2858,  -1.3038,  -1.3218],
         [  0.4821,   0.2824,   0.0826,  -0.1172],
         [ -0.1439,   0.0521,   0.2481,   0.4441],
         [ -1.4751,  -3.6797,  -5.8842,  -8.0888],
         [  2.1710,   4.3369,   6.5029,   8.6689],
         [ -1.4986,  -2.9875,  -4.4763,  -5.9652]],

        [[ -9.0803,  -9.2605,  -9.4407,  -9.6210],
         [  0.8929,  -1.1048,  -3.1025,  -5.1002],
         [ -1.3992,   0.5607,   2.5205,   4.4803],
         [-14.8788, -36.9243, -58.9699, -81.0154],
         [ 18.9396,  40.5993,  62.2589,  83.9186],
         [-15.1839, -30.0724, -44.9608, -59.8492]]])

In [23]:
p.shape

torch.Size([2, 6, 4])

In [24]:
targ = torch.tensor([[2, 4, 3, 1], [2, 1, -100, -100]])

In [25]:
loss = crit(p, targ)


In [26]:
loss

tensor([[  2.6276,   0.0351,  12.3911,   8.7865],
        [ 20.3388,  41.7041,   0.0000,   0.0000]])

In [155]:
lengths = np.array([2, 2, 1, 1])
length_tens = torch.from_numpy(lengths).float()

In [156]:
myzero1 = autograd.Variable(loss[1, 2], requires_grad=True)
myzeros2 = autograd.Variable(loss[1, 3], requires_grad=True)
myzero1.register_hook(print)
myzeros2.register_hook(print)

In [157]:
length_tens

tensor([2., 2., 1., 1.])

In [158]:
loss = loss.sum(dim=0) / length_tens

In [ ]:
loss

tensor([ 6.3544, 39.0284,  4.8090, 15.3148], grad_fn=<DivBackward1>)

In [161]:
loss.mean().backward()

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.